In [1]:
import requests
import json
import sqlite3
import numpy as np
import torch
from transformers import AutoTokenizer,AutoModel,DPRQuestionEncoder,DPRContextEncoder,RagTokenizer,RagSequenceForGeneration
from sentence_transformers import SentenceTransformer
import faiss
import re
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn.metrics import f1_score


### 数据获取 → 文本处理 → 向量编码 → 向量存储 → 检索模块 → 重排序模块 → 生成模块 → 评估模块

## 数据抓取

In [2]:
## 数据抓取
import re

class WikipediaCrawler:
    def __init__(self,topics,max_pages=20,max_section=5):
        self.base_url = "https://en.wikipedia.org/w/api.php"
        self.topics = topics
        self.max_pages = max_pages
        self.max_section = max_section
        self.stop_words = set(stopwords.words('english'))
        
    def clean_text(self,text):
        text = re.sub(r'\[\d+\]','',text) #移除引用
        text = re.sub(r'\s+',' ',text) # 移除多余的空格
        return text.strip() # 移除首尾空格
    
    def fetch_page_content(self,title):
        params = {
            'action':'query',
            'format':'json',
            'titles':title,
            'prop':'extracts', # 获取页面内容
            'explaintext':True,
            'redirects':True
        }
        response = requests.get(self.base_url,params=params).json()
        page = next(iter(response['query']['pages'].values()))
        return page['extract'] if 'extract' in page and page['extract'].strip() else None
    
    def fetch_links(self,topic):
        params = {
            'action':'query',
            'format':'json',
            'list':'search',
            'srsearch':topic, # 关键词
            'srlimit':self.max_pages # 结果数量
        }
        response = requests.get(self.base_url,params=params).json()
        return [item['title'] for item in response['query']['search']]
    
    def chunk_text(self,text,chunk_size=300,overlap=50):
        words =  text.split()
        chunks = []
        start = 0
        
        while start < len(words):
            end = min(start+chunk_size,len(words))
            chunk = ' '.join(words[start:end]) # 组合
            chunks.append(chunk)
            # 计算下一个块的起始位置，设置重叠区域
            start = end - overlap if end - overlap > start else end
        return chunks
    
    def run_crawl(self):
        all_data = []
        for topic in self.topics:
            links = self.fetch_links(topic)
            for link in tqdm(links[:self.max_pages],desc=f'Crawling {topic}'):
                content = self.fetch_page_content(link)
                if content:
                    cleaned = self.clean_text(content)
                    chunks = self.chunk_text(cleaned)
                    for i,chunk in enumerate(chunks[:self.max_section]):
                        all_data.append({
                            'id':f'{link}_{i}',
                            'title':link,
                            'topic':topic,
                            'text':chunk
                        })
        return all_data
    
    def expand_query(self,query):
        query = re.sub(r'[^\w\s]', '', query.lower())
        words = [w for w in query.split() if w not in self.stop_words]
        synonyms = {
            'capital': ['capital city', 'main city'],
            'developed': ['created', 'formulated', 'pioneered'],
            'france': ['french republic'],
            'relativity': ['theory of relativity', 'einstein theory'],
            'ai': ['artificial intelligence', 'machine intelligence']
        }
        expanded = []
        for word in words:
            expanded.append(word)
            if word in synonyms:
                expanded.extend(synonyms[word])
        
        # 去重并保留重要词汇
        return " ".join(sorted(set(expanded), key=len, reverse=True)[:5])


In [3]:
crawler = WikipediaCrawler(topics=["Artificial Intelligence",
                                    "Machine Learning", "Neural Networks"])
documents = crawler.run_crawl()

Crawling Neural Networks: 100%|████████████████████████████████████████████████████████| 20/20 [00:36<00:00,  1.81s/it]


In [4]:
documents

[{'id': 'Artificial intelligence_0',
  'title': 'Artificial intelligence',
  'topic': 'Artificial Intelligence',
  'text': 'Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals. High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI appli

## 数据处理与向量化

In [14]:
class VectorDatabase:
    def __init__(self,db_path='vector_store.db'):
        self.db_path = db_path
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        # 创建文档表格
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS documents\
        (id TEXT PRIMARY KEY, title TEXT, topic TEXT, text TEXT, embedding BLOB)''')
    
        self.conn.commit()
        
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        # 初始化，内积相似度
        self.index = faiss.IndexFlatIP(384) # 匹配模型的维度
        self.id_list = [] # 映射FAISS索引ID到文档ID
        self.rebuild_index() # 从数据库重建索引
        
    def rebuild_index(self):
        self.index.reset() # 清空索引
        self.id_list = []
        
        self.cursor.execute('SELECT rowid,id,embedding FROM documents')
        rows = self.cursor.fetchall()
        
        if not rows:
            print('dataset is empty-no index to rebuild')
            return

        
        embeddings = []
        for idx,(rowid,doc_id,emb_blob) in enumerate(rows):
            emb = np.frombuffer(emb_blob,dtype=np.float32)
            embeddings.append(emb)
            self.id_list.append(doc_id)
        embeddings = np.vstack(embeddings).astype('float32')
        self.index.add(embeddings)
        print(f'rebuilt index with {len(embeddings)} vectors')
        
        
    def add_documents(self,documents):
        texts = [doc['text'] for doc in documents]
        
        embeddings = self.model.encode(texts,
                                      show_progress_bar=True,
                                      batch_size=32)
        
        for doc,emb in zip(documents,embeddings):
            self.cursor.execute('INSERT OR REPLACE INTO documents VALUES (?, ?, ?, ?, ?)',
                               (doc['id'],doc['title'],doc['topic'],doc['text'],
                               emb.tobytes()))
            
        self.conn.commit()
        embeddings = np.array(embeddings).astype('float32')
        self.index.add(embeddings)
    
    def search(self,query,k=5):
        query_emb = self.model.encode([query])
        # FAISS搜索
        D,I = self.index.search(np.array(query_emb).astype('float32'),k)
        
        results = []
        for i,idx in enumerate(I[0]):
            if idx < 0:
                continue
            
            if idx < len(self.id_list):
                doc_id = self.id_list[idx]
                self.cursor.execute("SELECT * FROM documents WHERE id=?", (doc_id,))
                row = self.cursor.fetchone()
            
                if row:
                    results.append({
                        'id':row[0],
                        'title':row[1],
                        'topic':row[2],
                        'text':row[3],
                        'score':D[0][i] # 相似度分数
                    })
            else:
                print(f'warning:index {idx} out of range (max{len(self.id_list)-1})')
        return results
    
    def get_document(self,doc_id):
        self.cursor.execute("SELECT * FROM documents WHERE id=?", (doc_id,))
        row = self.cursor.fetchone()
        if row:
            return {
                "id": row[0],
                "title": row[1],
                "topic": row[2],
                "text": row[3]
            }
        return None
    
    def index_status(self):
        """检查索引状态"""
        print(f"\nIndex contains {self.index.ntotal} vectors")
        print(f"Database has {self.get_document_count()} documents")
        
    def get_document_count(self):
        """获取文档数量"""
        self.cursor.execute("SELECT COUNT(*) FROM documents")
        return self.cursor.fetchone()[0] 

## RAG

In [36]:
from transformers import AutoModelForSequenceClassification
from transformers import RagTokenForGeneration
class RAGSystem:
    def __init__(self,db_path = 'vector_store.db'):
        self.db = VectorDatabase(db_path)
    
        # 检索模型： DPR
        self.retriever_model = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
        self.retriever_tokenizer = AutoTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
    
        # 生成模型 ： RAG-Sequence

        self.generator_model = RagTokenForGeneration.from_pretrained("facebook/rag-token-nq")
        self.generator_tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
    
        # 重排序模型 ： cross-encoder
        self.rerank_model = AutoModelForSequenceClassification.from_pretrained("cross-encoder/ms-marco-MiniLM-L-6-v2")
        self.rerank_tokenizer = AutoTokenizer.from_pretrained("cross-encoder/ms-marco-MiniLM-L-6-v2")
        
    def retriever(self,query,k=10):
        # 第一阶段检索
        expanded_query = self.expand_query(query)
        print(f'[DEBUG] Expanded query: {expanded_query}')
        
        results = self.db.search(expanded_query, k)
        print(f'[DEBUG] Retrieved {len(results)} documents')
        
        if results:
            for i, doc in enumerate(results[:3]):
                print(f'  Doc {i+1}: {doc["title"]} (score: {doc["score"]:.3f})')
                print(f'  Text: {doc["text"][:80]}...')
        
        return results
    
    def expand_query(self, query):
        synonyms = {
            'capital': ['capital city', 'main city', 'administrative center'],
            'developed': ['created', 'formulated', 'pioneered', 'invented'],
            'france': ['french republic', 'french nation'],
            'relativity': ['theory of relativity', 'einstein theory', 'space-time theory'],
            'ai': ['artificial intelligence', 'machine intelligence', 'cognitive computing'],
            'what': ['define', 'explain', 'describe'],
            'who': ['scientist', 'physicist', 'researcher']
        }
    
    # 添加更智能的扩展
        expanded = []
        for word in query.lower().split():
            if word in synonyms:
                expanded.extend(synonyms[word])
            else:
                expanded.append(word)
    
        unique_terms = list(set(expanded))
        return " ".join(unique_terms)
    
    def rerank(self,query,documents):
        # 过滤空的文档
        valid_docs = [doc for doc in documents if doc.get('text','').strip()]
        if not valid_docs:
            return []
        
        pairs = [(query,doc['text']) for doc in valid_docs]
        
        # 编码
        features = self.rerank_tokenizer(pairs,
                                        padding=True,
                                        truncation=True,
                                        return_tensors='pt',
                                        max_length=512)
        # 计算相关性分数
        with torch.no_grad():
            outputs = self.rerank_model(**features)
            scores = outputs.logits.squeeze(dim=1)
            
        # 更新文档分数
        for i,doc in enumerate(documents):
            doc['rerank_score'] = scores[i].item() if i < len(scores) else 0.0
            
        return sorted(documents,key=lambda x: x['rerank_score'],reverse=True)
    
    def generate(self,query,context_docs,max_length=150):
        if not context_docs:
            print("[WARNING] No context documents for generation")
            return 'No relevant context found'
        
        context_texts = [doc['text'] for doc in context_docs]
        
        inputs = self.generator_tokenizer.prepare_seq2seq_batch(query, # question
                                           context_texts, # 上下文列表
                                          return_tensors='pt',
                                        )
        # 生成结果
        outputs = self.generator_model.generate(input_ids=inputs['input_ids'],
                                               attention_mask=inputs['attention_mask'],
                                               max_length=max_length,
                                               num_beams=4,
                                               early_stopping=True,
                                               no_repeat_ngram_size=3,# 避免重复
                                               length_penalty=0.8# 长度惩罚
                                               )
        
        return self.generator_tokenizer.decode(outputs[0],
                                              skip_special_tokens=True)
    
    def full_pipeline(self,query):
        # 检索
        retrieved = self.retriever(query,k=10)
        
        # 重排序-- 精排
        reranked = self.rerank(query,retrieved)[:3] # 取top3
        
        # 生成答案
        return self.generate(query,reranked)
    

## 评估

In [37]:
class RAGEvaluator:
    def __init__(self,rag_system):
        self.rag = rag_system
        # 自己构建测试集来判断
        self.dataset = [
            {
                "question": "What is the capital of France?",
                "answer": "Paris",
                "context": ["France is a country in Europe. Its capital is Paris."]
            },
            {
                "question": "Who developed the theory of relativity?",
                "answer": "Albert Einstein",
                "context": ["Albert Einstein was a physicist who developed the theory of relativity."]
            },
            {
                "question": "What is AI?",
                "answer": "Artificial Intelligence",
                "context": ["Artificial intelligence (AI) is the simulation of human intelligence processes by machines."]
            }
        ]
        
    def exact_match(self,pred,true):
        pred = pred.lower().strip()
        true = true.lower().strip()
        return int(pred==true)
    
    def f1_score(self,pred,true):
        pred_tokens = set(pred.lower().split())
        true_tokens = set(true.lower().split())
        
        if not pred_tokens or not true_tokens:
            return 0.0
        
        # 计算交集
        common = pred_tokens & true_tokens
        
        precision = len(common) / len(pred_tokens) if pred_tokens else 0
        recall = len(common) / len(true_tokens) if true_tokens else 0
        
        if precision + recall == 0:
            return 0.0
        
        return 2 * (precision * recall) / (precision + recall)
    
    def context_relevance(self,retrieved,gold_context):
        # 上下文评估
        gold_text = ' '.join(gold_context).lower()
        gold_token = set(gold_text.split())
        max_overlap = 0
        
        for doc in retrieved:
            doc_text = doc['text'].lower()
            doc_token = set(doc_text.split())
            # 计算雅可比相似度
            overlap = len(gold_token & doc_token)
            similarity = overlap / len(gold_token) if gold_token else 0
            if similarity > max_overlap:
                max_overlap = similarity
                
        return max_overlap
    
    def evaluate(self):
        results = []
        for item in tqdm(self.dataset,desc='Evaluating RAG'):
            
            answer = self.rag.full_pipeline(item['question'])
        
        # 单独检索
            retrieved = self.rag.retriever(item['question'])
        
        # 计算指标
            em = self.exact_match(answer,item['answer']) # 完全匹配率
            f1 = self.f1_score(answer,item['answer']) # 词重叠的相似度
            ctx_rel = self.context_relevance(retrieved,item['context']) # 检索相关性
        
            results.append({
                    "question": item["question"],
                    "predicted": answer,
                    "expected": item["answer"],
                    "em": em,
                    "f1": f1,
                    "context_relevance": ctx_rel
                })
        
        avg_em = sum(r["em"] for r in results) / len(results)
        avg_f1 = sum(r["f1"] for r in results) / len(results)
        avg_ctx = sum(r["context_relevance"] for r in results) / len(results)
        
        return {
            "results": results,  # 详细结果
            "metrics": {  # 平均指标
                "exact_match": avg_em,
                "f1_score": avg_f1,
                "context_relevance": avg_ctx
            }
        }

In [ ]:
from ipywidgets import widgets, Layout
from IPython.display import display
import os
# 使用示例
if __name__ == "__main__":
    # 示例文档数据

    documents_1 = [
        {
            "id": "1",
            "title": "France",
            "topic": "Geography",
            "text": "France is a country in Europe. Its capital is Paris."
        },
        {
            "id": "2",
            "title": "Einstein",
            "topic": "Science",
            "text": "Albert Einstein was a physicist who developed the theory of relativity."
        },
        {
            "id": "3",
            "title": "Artificial Intelligence",
            "topic": "Technology",
            "text": "Artificial intelligence (AI) is the simulation of human intelligence processes by machines, especially computer systems. Specific applications of AI include expert systems, natural language processing, speech recognition and machine vision."
        },
        {
            "id": "4",
            "title": "Machine Learning",
            "topic": "Technology",
            "text": "Machine learning (ML) is a type of artificial intelligence that allows software applications to become more accurate at predicting outcomes without being explicitly programmed to do so. Machine learning algorithms use historical data as input to predict new output values."
        }
    ]
    # 初始化向量数据库
    vector_db = VectorDatabase()
    vector_db.add_documents(documents_1)
    
    print(f"Database contains {vector_db.get_document_count()} documents")
    
    print("\nTesting search function...")
    test_queries = ["capital of france", "who created relativity", "define AI"]
    for query in test_queries:
        print(f"\nQuery: '{query}'")
        results = vector_db.search(query, k=2)
        print(f"Found {len(results)} results")
        for res in results:
            print(f"- {res['title']} (score: {res['score']:.3f})")
    
    print("\nInitializing RAG system...")
    rag_system = RAGSystem()
    
    print("\nTesting full pipeline...")
    questions = [
        "What is the capital of France?",
        "Who developed the theory of relativity?",
        "What is AI?"
    ]
    for q in questions:
        print(f"\nQuestion: {q}")
        answer = rag_system.full_pipeline(q)
        print(f"Answer: {answer}")
    
    print("\nRunning evaluation...")
    evaluator = RAGEvaluator(rag_system)
    eval_results = evaluator.evaluate()
    
    print("\nEvaluation Metrics:")
    for metric, value in eval_results["metrics"].items():
        print(f"{metric.replace('_', ' ').title()}: {value:.4f}")